In [ ]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import json

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
chrome_options = Options()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument("--disable-xss-auditor")
chrome_options.add_argument("--disable-web-security")
chrome_options.add_argument("--allow-running-insecure-content")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument("--disable-webgl")
chrome_options.add_argument("--disable-popup-blocking")

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
url = 'https://agbrief.com/news/malaysia'
driver.get(url)

In [ ]:
#guna selenium utk autom scrolling web page, sebab depa guna load more bila
#scroll, kira infinite scroll la kot, tapi ada hujung dia gak

scroll_counter = 0
while(True):
  driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
  scroll_counter += 1
  time.sleep(3)
  if scroll_counter > 20:
    break

In [ ]:
#lepas habih scroll semua, collect semua link yang surface

links = []
href = driver.find_elements(By.TAG_NAME, "a")

for i in tqdm(range(len(href))):
  links.append(href[i].get_attribute("href"))

links

100%|██████████████████████████████████████████████████████████████████████████████| 1186/1186 [00:19<00:00, 60.48it/s]


['https://www.facebook.com/AsiaGamingBrief/',
 'https://www.instagram.com/asiagamingbrief/',
 'https://linkedin.com/company/agbrief',
 'https://agbrief.com/category/news/feed/',
 'https://soundcloud.com/agbrief',
 'https://twitter.com/agbrief',
 'https://youtube.com/channel/UCOfKf4dNjZtiV7mXwC8oU4A',
 None,
 'https://agbrief.com/',
 'https://agbrief.com/intelligence/',
 'https://agbrief.com/category/intelligence/face-to-face/',
 'https://agbrief.com/category/intelligence/deep-dive/',
 'https://agbrief.com/category/intelligence/infographics/',
 'https://www.asiagamingdata.com/',
 'https://agbrief.com/category/intelligence/podcast/',
 'https://agbrief.com/headlines/',
 'https://agbrief.com/tag/industry_updates/',
 'https://agbrief.com/magazines/',
 'https://agbrief.com/news/malaysia/#',
 'https://agbrief.com/events/',
 'https://aseangaming.com/',
 'https://asiagamingawards.com/',
 'https://agbrief.com/gaming-webinars/',
 'https://mekonggamingsummit.com/',
 'https://www.facebook.com/Asian

In [ ]:
len(links)

1186

In [ ]:
#the extracted links may contain NoneType, and the links may duplicate
#unique links

def extract_unique_strings(input_list):
    string_set = set()
    for item in tqdm(input_list):
        if isinstance(item, str):
            string_set.add(item)
    return list(string_set)

unique_links = extract_unique_strings(links)
len(unique_links)

100%|█████████████████████████████████████████████████████████████████████████| 1186/1186 [00:00<00:00, 1174603.20it/s]


342

In [ ]:
#filter lagi unique links tu filtered to news/malaysia/...

import re
malaysia_links = [url for url in unique_links if re.match(r'^https://agbrief\.com/news/malaysia', url)]

len(malaysia_links)

190

In [ ]:
malaysia_links[0]

'https://agbrief.com/news/malaysia/05/11/2020/genting-outlook-cut-to-negative-by-fitch/'

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36',
           'Referer':'https://aliffchannel.com/'}

In [ ]:
#function untuk scrape text, guna keyword 'p' direct sebab ada non body content
#guna class yang sama, so serabut gak mikir. bila guna keyword 'p' ni, ada termasuk
#yang kita takmau, tapi every links sama pattern, start lepas string SUBMIT.. tu
#bukan body content dah.

def scrape_content(good_soup):

  ls =[]
  content = good_soup.find_all('p')

  for x in content:
    if x.text == 'SUBMIT YOUR ENQUIRY HERE':
      break

    ls.append(x.text)
  combine_string = "".join(ls)
  return combine_string


In [ ]:
with open('agbrief-1.jsonl', 'a') as f:
    for a in tqdm(malaysia_links):
        try:
            r = requests.get(a, headers=headers)
            soup = BeautifulSoup(r.text, "html")
            body = scrape_content(soup)
            data = {
                'url': a,
                'title': soup.find('h1', attrs = {"class":"tdb-title-text"}).get_text(),
                'body': body,
            }
            json.dump(data, f)
            f.write('\n')
        except:
            pass

100%|████████████████████████████████████████████████████████████████████████████████| 190/190 [13:45<00:00,  4.35s/it]
